In [1]:
import pandas as pd
import random
import sys
sys.path.append('../..')

from model import run
from model import plot_utils as pu
from model import processing
from model.amm.omnipool_amm import OmnipoolState
from model.amm.stableswap_amm import StableSwapPoolState
from model.amm.agents import Agent
from model.amm.trade_strategies import random_swaps
from model.amm.global_state import GlobalState, swap, add_liquidity, migrate
from model.amm import omnipool_amm as oamm

# same seed, same parameters = same simulation result
random.seed(42)

price_dict = {
    'HDX': 0.05,
    'USD': 1,
    'LRNA': 0.5,
    'R1': 2,
    'R2': 2/3,
    'AUSD': 1.0000001
}

def lrna_price(asset: str):
    return price_dict['LRNA'] / price_dict[asset]

# set up the state

initial_state = GlobalState(
    pools = {
        "Omnipool": OmnipoolState(
            tokens={
                'HDX': {'liquidity': 1000000, 'LRNA_price': lrna_price('HDX')},
                'USD': {'liquidity': 1000000, 'LRNA_price': lrna_price('USD')},
                'AUSD': {'liquidity': 1000000, 'LRNA_price': lrna_price('AUSD')},
                'R1': {'liquidity': 1000000, 'LRNA_price': lrna_price('R1')},
                'R2': {'liquidity': 3000000, 'LRNA_price': lrna_price('R2')},
            },
            lrna_fee=0,  # these can also be FeeMechanism type
            asset_fee=0,
            sub_pools={
                'Stableswap': StableSwapPoolState(
                    tokens={
                        'DAI': 1000000,
                        'TETHER': 1000000
                    },
                    amplification=100,
                    precision=0.00001,
                    trade_fee=0.003  
                )
            }
        )
    },
    agents = {
        'Trader': Agent(
            holdings={'HDX': 10000, 'USD': 10000, 'R1': 10000, 'R2': 10000},
        ),
        'LP': Agent(
            holdings={'AUSD': 1000000}, 
        )
    }
)

In [1]:
# sell an asset
buy_quantity = 1000
tkn_buy = 'DAI'
tkn_sell = 'R1'

buy_state = swap(
    old_state=initial_state,
    pool_id='Omnipool',
    agent_id='Trader',
    tkn_buy=tkn_buy,
    tkn_sell=tkn_sell,
    buy_quantity=buy_quantity
)
print(buy_state)

global state 
pools: 

    Omnipool: Omnipool
    ********************************
    tvl cap: inf
    lrna fee: 0
    asset fee: 0
    asset pools: (
    
        *HDX*
        asset quantity: 1000000.0
        lrna quantity: 100000.0
        USD price: 0.05
        tvl: 50000.0
        weight: 100000.0/16100000.2 (0.0062111800470660864)
        weight cap: 1.0
        total shares: 1000000.0
        protocol shares: 1000000.0
    
        *USD*
        asset quantity: 1000000.0
        lrna quantity: 2000000.0
        USD price: 1.0
        tvl: 1000000.0
        weight: 2000000.0/16100000.2 (0.12422360094132173)
        weight cap: 1.0
        total shares: 1000000.0
        protocol shares: 1000000.0
    
        *AUSD*
        asset quantity: 1000000.0
        lrna quantity: 2000000.2000000002
        USD price: 1.0000001
        tvl: 1000000.1000000001
        weight: 2000000.2000000002/16100000.2 (0.12422361336368185)
        weight cap: 1.0
        total shares: 1000000.0
    

In [2]:
# reverse that operation
sell_quantity = initial_state.agents['Trader'].holdings[tkn_sell] - buy_state.agents['Trader'].holdings[tkn_sell]
sell_state = swap(
    old_state=initial_state,
    pool_id='Omnipool',
    agent_id='Trader',
    tkn_buy=tkn_buy,
    tkn_sell=tkn_sell,
    sell_quantity=sell_quantity
)
print(sell_state)

global state 
pools: 

    Omnipool: Omnipool
    ********************************
    tvl cap: inf
    lrna fee: 0
    asset fee: 0
    asset pools: (
    
        *HDX*
        asset quantity: 1000000.0
        lrna quantity: 100000.0
        USD price: 0.05
        tvl: 50000.0
        weight: 100000.0/16100000.2 (0.0062111800470660864)
        weight cap: 1.0
        total shares: 1000000.0
        protocol shares: 1000000.0
    
        *USD*
        asset quantity: 1000000.0
        lrna quantity: 2000000.0
        USD price: 1.0
        tvl: 1000000.0
        weight: 2000000.0/16100000.2 (0.12422360094132173)
        weight cap: 1.0
        total shares: 1000000.0
        protocol shares: 1000000.0
    
        *AUSD*
        asset quantity: 1000000.0
        lrna quantity: 2000000.2000000002
        USD price: 1.0000001
        tvl: 1000000.1000000001
        weight: 2000000.2000000002/16100000.2 (0.12422361336368185)
        weight cap: 1.0
        total shares: 1000000.0
    

In [3]:
# chain operations together
partial_lp_state = add_liquidity(
    old_state=sell_state,
    pool_id='Omnipool',
    agent_id='LP',
    tkn_add='AUSD',
    quantity=500000
)
migrate_state = migrate(
    old_state=partial_lp_state,
    pool_id='Omnipool',
    sub_pool_id='Stableswap',
    tkn_migrate='AUSD'
)
more_lp_state = add_liquidity(
    old_state=migrate_state,
    pool_id='Omnipool',
    agent_id='LP',
    tkn_add='AUSD',
    quantity=500000
)
print(more_lp_state)

global state 
pools: 

    Omnipool: Omnipool
    ********************************
    tvl cap: inf
    lrna fee: 0
    asset fee: 0
    asset pools: (
    
        *HDX*
        asset quantity: 1000000.0
        lrna quantity: 100000.0
        USD price: 0.05
        tvl: 50000.0
        weight: 100000.0/17100000.3 (0.0058479531137786)
        weight cap: 1.0
        total shares: 1000000.0
        protocol shares: 1000000.0
    
        *USD*
        asset quantity: 1000000.0
        lrna quantity: 2000000.0
        USD price: 1.0
        tvl: 1000000.0
        weight: 2000000.0/17100000.3 (0.116959062275572)
        weight cap: 1.0
        total shares: 1000000.0
        protocol shares: 1000000.0
    
        *R1*
        asset quantity: 1000000.0
        lrna quantity: 4000000.0
        USD price: 2.0
        tvl: 2000000.0
        weight: 4000000.0/17100000.3 (0.233918124551144)
        weight cap: 1.0
        total shares: 1000000.0
        protocol shares: 1000000.0
    
      